In [1]:
import numpy as np
import cv2 as cv
from cake import cake
from matplotlib import pyplot as plt
from cluster_half_shelfs_alpha import Half_shelf_cluster
from confectionery import Confectionery
from particle_data_structure import ParticleDataStruct
from scipy.spatial.transform import Rotation as R
from utitlity_functions import *
import os


# input parameter

In [2]:
folder_name="Gute_aufnahme_shortened"


In [3]:
data_paths=os.listdir(os.path.join("data",folder_name))
stream_string=np.load(os.path.join("data",folder_name,"image_paths.npy"))
objs=[x for x in data_paths if "pos_data" in x]
objs_number=np.array([int(x[9:-4]) for x in objs])
indexing=np.argsort(objs_number)
objs=[np.load(os.path.join("data",folder_name,objs[x])) for x in indexing]
number_objs=len(objs)

koppenrath=Confectionery()
particle_structured3d_path=os.path.join("data",folder_name,"particle_structured3d_")
particle_structured_flat_path=os.path.join("data",folder_name,"particle_structured_flat_")
cut_radius=[int(np.floor(np.mean(x[:,-1])))-1 for x in objs]
structured_datas=[ParticleDataStruct(x,stream_string,c,koppenrath) for x,c in zip(objs,cut_radius)]
particle_datas_3d=[np.load(particle_structured3d_path+str(i)+".npy")for i in range(number_objs)]
particle_datas_flat=[np.load(particle_structured_flat_path+str(i)+".npy") for i in range(number_objs)]
particle_datas_flat_effective=[x.get_data_flatten_effective(c,y) for x,y,c in zip(structured_datas,particle_datas_flat,cut_radius)]

In [4]:
# set frames per second
fps=15

# object number
object_number=11

# destination video object segmented
destination_video_segmented = os.path.join("Video_Data",folder_name,"normalized_segmented_particle_")+str(object_number)+'.avi'
# destination video clustering
destination_video_clustering=os.path.join("Video_Data",folder_name,"clustered_particle_")+str(object_number)+".avi"
"""
# destination arrowed clustering 
destination_video_arrowed_clustering = '/Users/Hoang_1/Desktop/Master_Arbeit/software/janus_particle_tracking/Video_Data/object_'+str(object_number)+'_clustered_arrows_only_true.avi'
# destination double arrowed clustering 
destination_video_two_arrowed_clustering= '/Users/Hoang_1/Desktop/Master_Arbeit/software/janus_particle_tracking/Video_Data/object_'+str(object_number)+'_clustered_arrows.avi'

# rotation path
rotation_trajectory_rot_vec=np.load("data/rotation_trajectory_as_rotvec_"+str(object_number)+".npy")
"""




'\n# destination arrowed clustering \ndestination_video_arrowed_clustering = \'/Users/Hoang_1/Desktop/Master_Arbeit/software/janus_particle_tracking/Video_Data/object_\'+str(object_number)+\'_clustered_arrows_only_true.avi\'\n# destination double arrowed clustering \ndestination_video_two_arrowed_clustering= \'/Users/Hoang_1/Desktop/Master_Arbeit/software/janus_particle_tracking/Video_Data/object_\'+str(object_number)+\'_clustered_arrows.avi\'\n\n# rotation path\nrotation_trajectory_rot_vec=np.load("data/rotation_trajectory_as_rotvec_"+str(object_number)+".npy")\n'

In [5]:
# load object trajectory and their correspondent image paths
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
start_frame=round(objs[object_number][0,0])
end_frame=round(objs[object_number][-1,0])

obj=objs[object_number]

half_shelf_cluster=Half_shelf_cluster(obj, stream_string, confectionery=koppenrath,cut_radius=cut_radius[object_number])
half_shelf_cluster.load_data_from_np_array((particle_datas_flat_effective[object_number].flatten()))

In [6]:
# set frames per second
fps=15
# set annotation
annotate=True
# duplication of pixels
b_length=10

In [7]:

radius=cut_radius[object_number]
video_name = destination_video_segmented
height=b_length*(2*(radius)+1)
width=height
video = cv.VideoWriter(video_name, 0, fps, (height,width),isColor=True)
# for writing on image
font = cv.FONT_HERSHEY_SIMPLEX
org = (20, 25)
fontScale = 1
color = (255, 0, 0)
thickness = 1
B=np.ones((b_length,b_length))
M=koppenrath.sell_cake(radius,2*radius+1,2*radius+1)
for current_frame_number in range(start_frame,end_frame+1):
    obj_image_current=cv.imread(stream_string[current_frame_number])
    y_pos=obj[current_frame_number-start_frame,2]
    x_pos=obj[current_frame_number-start_frame,1]
    object_indeces=np.s_[round(y_pos-radius):round(y_pos+radius)+1,
                        round(x_pos-radius):round(x_pos+radius)+1,:]
    obj_image_current=obj_image_current[object_indeces]
    # normalization
    obj_image_current=cv.cvtColor(obj_image_current,cv.COLOR_BGR2GRAY)
    v_min=np.min(obj_image_current[M==1])
    v_max=np.max(obj_image_current[M==1])
    obj_image_current[obj_image_current>v_max]=v_max
    obj_image_current=obj_image_current-v_min
    obj_image_current=obj_image_current.astype(np.float64)
    obj_image_current=255*obj_image_current/(v_max-v_min)
    obj_image_current=obj_image_current.astype(np.uint8)
    obj_image_current=cv.cvtColor(obj_image_current,cv.COLOR_GRAY2BGR)

    obj_image_current = cv.circle(obj_image_current, (round(x_pos),round(y_pos)), radius=0, color=(0, 0, 255), thickness=-1)
    temp_im=np.zeros((obj_image_current.shape[0]*b_length,obj_image_current.shape[1]*b_length,obj_image_current.shape[2]),np.uint8)
    for i in range(3):
        temp_im[:,:,i]=np.kron(obj_image_current[:,:,i],B)
    if annotate==True:
        temp_im=cv.putText(temp_im, '#'+str(current_frame_number).zfill(3), org, font, 
                    fontScale, color, thickness, cv.LINE_AA)
    video.write(temp_im)
cv.destroyAllWindows()
video.release()



# Clustering and tracking next to each other

In [8]:
radius=cut_radius[object_number]
b_length=10
video_name = destination_video_clustering
height=b_length*(2*(radius)+1)
width=2*height
#width=height
video = cv.VideoWriter(video_name, 0, fps, (width,height),isColor=True)
# for writing on image
font = cv.FONT_HERSHEY_SIMPLEX
org = (20, 25)
fontScale = 1
color = (255, 0, 0)
thickness = 1
blue_vector=np.reshape(np.array([255,0,0]),(1,1,3))
green_vector=np.reshape(np.array([0,128,0]),(1,1,3))
B=np.ones((b_length,b_length))
M=koppenrath.sell_cake(radius,2*radius+1,2*radius+1)

for current_frame_number in range(start_frame,end_frame):
    obj_image_current=cv.imread(stream_string[current_frame_number])
    y_pos=obj[current_frame_number-start_frame,2]
    x_pos=obj[current_frame_number-start_frame,1]
    object_indeces=np.s_[round(y_pos-radius):round(y_pos+radius)+1,
                        round(x_pos-radius):round(x_pos+radius)+1,:]
    obj_image_current=obj_image_current[object_indeces]
    # normalization
    obj_image_current=cv.cvtColor(obj_image_current,cv.COLOR_BGR2GRAY)
    obj_image_current_gray=np.copy(obj_image_current)
    v_min=np.min(obj_image_current[M==1])
    v_max=np.max(obj_image_current[M==1])
    obj_image_current[obj_image_current>v_max]=v_max
    obj_image_current=obj_image_current-v_min
    obj_image_current=obj_image_current.astype(np.float64)
    obj_image_current=255*obj_image_current/(v_max-v_min)
    obj_image_current=obj_image_current.astype(np.uint8)
    obj_image_current=cv.cvtColor(obj_image_current,cv.COLOR_GRAY2BGR)

    # cut border
    image_shape=obj_image_current.shape[:2]
    object_pixels=np.copy(obj_image_current_gray)
    object_pixels=np.float32(object_pixels[M==1])
    label=half_shelf_cluster.label(object_pixels.reshape(-1),method="GMM")
    label=label+1
    object_clustered=np.zeros(image_shape,np.uint8)
    object_clustered[M==1]=label.reshape(-1)



    temp_clustered=np.zeros(obj_image_current.shape,np.uint8)
    for l in range(image_shape[0]):
        for k in range(image_shape[1]):
            if object_clustered[l,k]==1:
                temp_clustered[l,k,:]=blue_vector
            elif object_clustered[l,k]==2:
                temp_clustered[l,k,:]=green_vector
    object_clustered=temp_clustered
    #pad arrays until they have suitable shape
    #pad_length=round(max_radius-radius)
    temp_1=np.zeros((2*(radius)+1,2*(radius)+1,3),np.uint8)
    temp_2=np.copy(temp_1)
    
    for i in range(3):
        temp_1[:,:,i]=obj_image_current[:,:,i]
        temp_2[:,:,i]=object_clustered[:,:,i]
    
    temp_im_1=np.zeros((temp_1.shape[0]*b_length,temp_1.shape[1]*b_length,3),np.uint8)
    temp_im_2=np.copy(temp_im_1)
    for i in range(3):
        temp_im_1[:,:,i]=np.kron(temp_1[:,:,i],B)
        temp_im_2[:,:,i]=np.kron(temp_2[:,:,i],B)
    temp_im=np.zeros((height,width,3),np.uint8)
    temp_im[:,:temp_im_1.shape[0],:]=temp_im_1
    temp_im[:,temp_im_1.shape[0]:,:]=temp_im_2
    if annotate==True:
        temp_im=cv.putText(temp_im, '#'+str(current_frame_number).zfill(3), org, font, 
                    fontScale, color, thickness, cv.LINE_AA)
    video.write(np.uint8(temp_im))
cv.destroyAllWindows()
video.release()


# TODO!!!!!!!!!!!

# Make Video true directions

In [9]:
max_radius=round(np.ceil(np.max(obj[:,-1])))
V_true=obj[1:,1:3]-obj[:-1,1:3]
dist_true=np.linalg.norm(V_true,axis=1)
V_true=np.array([v/np.linalg.norm(v) if np.linalg.norm(v)>1e-6 else np.zeros(2) for v in V_true])
norm_const=max_radius/np.mean(dist_true)


b_length=10
video_name = destination_video_arrowed_clustering
height=b_length*(2*(max_radius)+1)
width=2*height
#width=height
video = cv.VideoWriter(video_name, 0, fps, (width,height),isColor=True)
# for writing on image
font = cv.FONT_HERSHEY_SIMPLEX
org = (20, 25)
fontScale = 1
color = (255, 0, 0)
thickness = 1
blue_vector=np.reshape(np.array([255,0,0]),(1,1,3))
green_vector=np.reshape(np.array([0,128,0]),(1,1,3))
B=np.ones((b_length,b_length))
for current_frame_number in range(start_frame,end_frame):
    # get object image part
    obj_image_current=cv.imread(stream_string[current_frame_number])
    y_pos=obj[current_frame_number-start_frame,2]
    x_pos=obj[current_frame_number-start_frame,1]
    radius=np.ceil(obj[current_frame_number-start_frame,3])
    object_indeces=np.s_[round(y_pos-radius):round(y_pos+radius)+1,
                        round(x_pos-radius):round(x_pos+radius)+1,:]
    obj_image_current=obj_image_current[object_indeces]
    obj_image_current_gray=cv.cvtColor(obj_image_current,cv.COLOR_RGB2GRAY)

    arrow_length = dist_true[current_frame_number-start_frame]*norm_const
    if arrow_length>max_radius:
        arrow_length=max_radius
    
    v=V_true[current_frame_number-start_frame,:]
    start_point = (obj_image_current.shape[0] // 2, obj_image_current.shape[0] // 2)
    end_point = (start_point[0] + int(v[0] * arrow_length), start_point[1] + int(v[1] * arrow_length))
    obj_image_current=cv.arrowedLine(obj_image_current, start_point, end_point, (255,0,0), 1)
    

    # cut border
    image_shape=obj_image_current_gray.shape
    cut_radius=r_effective
    M=koppenrath.sell_cake(r_effective,obj_image_current_gray.shape[0],obj_image_current_gray.shape[0])
    object_pixels=np.copy(obj_image_current_gray)
    object_pixels=np.float32(object_pixels[M==1])
    label=half_shelf_cluster.label(object_pixels.reshape(-1),method="GMM")
    label=label+1
    object_clustered=np.zeros(image_shape,np.uint8)
    object_clustered[M==1]=label.reshape(-1)



    temp_clustered=np.zeros(obj_image_current.shape,np.uint8)
    for l in range(image_shape[0]):
        for k in range(image_shape[1]):
            if object_clustered[l,k]==1:
                temp_clustered[l,k,:]=blue_vector
            elif object_clustered[l,k]==2:
                temp_clustered[l,k,:]=green_vector
    object_clustered=temp_clustered

    #pad arrays until they have suitable shape
    pad_length=round(max_radius-radius)
    temp_1=np.zeros((2*(max_radius)+1,2*(max_radius)+1,3),np.uint8)
    temp_2=np.copy(temp_1)
    for i in range(3):
        temp_1[:,:,i]=np.pad(obj_image_current[:,:,i], pad_width=[(pad_length,pad_length),(pad_length,pad_length)], mode='constant')
        temp_2[:,:,i]=np.pad(object_clustered[:,:,i], pad_width=[(pad_length,pad_length),(pad_length,pad_length)], mode='constant')

    temp_im_1=np.zeros((temp_1.shape[0]*b_length,temp_1.shape[1]*b_length,3),np.uint8)
    temp_im_2=np.copy(temp_im_1)
    for i in range(3):
        temp_im_1[:,:,i]=np.kron(temp_1[:,:,i],B)
        temp_im_2[:,:,i]=np.kron(temp_2[:,:,i],B)
    temp_im=np.zeros((height,width,3),np.uint8)
    temp_im[:,:temp_im_1.shape[0],:]=temp_im_1
    temp_im[:,temp_im_1.shape[0]:,:]=temp_im_2
    if annotate==True:
        temp_im=cv.putText(temp_im, '#'+str(current_frame_number).zfill(3), org, font, 
                    fontScale, color, thickness, cv.LINE_AA)
    video.write(np.uint8(temp_im))
cv.destroyAllWindows()
video.release()


NameError: name 'destination_video_arrowed_clustering' is not defined

# Make Video True directions and calculated directions

In [ ]:
V_rot=np.copy(rotation_trajectory_rot_vec[:,[1,0]])
V_rot[:,1]=-V_rot[:,1]
normalization_factor=np.linalg.norm(V_rot,axis=1)
normalization_factor[normalization_factor<1e-6]=1
V_rot=V_rot/normalization_factor[:,np.newaxis]

r=np.mean(obj[:,-1])
p_0=np.array([0,0,1])
rotation_trajectory_dict=dict([[j,R.from_rotvec(r)] for j,r in enumerate(rotation_trajectory_rot_vec)])
dist_rot=distance_rot_on_sphere(p_0,rotation_trajectory_dict,r)
dist_rot=dist_rot[1:]-dist_rot[:-1]

b_length=10
video_name = destination_video_two_arrowed_clustering
height=b_length*(2*(max_radius)+1)
width=2*height
#width=height
video = cv.VideoWriter(video_name, 0, fps, (width,height),isColor=True)
# for writing on image
font = cv.FONT_HERSHEY_SIMPLEX
org = (20, 25)
fontScale = 1
color = (255, 0, 0)
thickness = 1
blue_vector=np.reshape(np.array([255,0,0]),(1,1,3))
green_vector=np.reshape(np.array([0,128,0]),(1,1,3))
B=np.ones((b_length,b_length))
for current_frame_number in range(start_frame,end_frame):
    # get object image part
    obj_image_current=cv.imread(stream_string[current_frame_number])
    y_pos=obj[current_frame_number-start_frame,2]
    x_pos=obj[current_frame_number-start_frame,1]
    radius=np.ceil(obj[current_frame_number-start_frame,3])
    object_indeces=np.s_[round(y_pos-radius):round(y_pos+radius)+1,
                        round(x_pos-radius):round(x_pos+radius)+1,:]
    obj_image_current=obj_image_current[object_indeces]
    obj_image_current_gray=cv.cvtColor(obj_image_current,cv.COLOR_RGB2GRAY)

    arrow_length = dist_true[current_frame_number-start_frame]*norm_const
    if arrow_length>max_radius:
        arrow_length=max_radius
    
    v=V_true[current_frame_number-start_frame,:]
    start_point = (obj_image_current.shape[0] // 2, obj_image_current.shape[0] // 2)
    end_point = (start_point[0] + int(v[0] * arrow_length), start_point[1] + int(v[1] * arrow_length))
    obj_image_current=cv.arrowedLine(obj_image_current, start_point, end_point, (255,0,0), 1)
    
    arrow_length = dist_rot[current_frame_number-start_frame]*norm_const
    if arrow_length>max_radius:
        arrow_length=max_radius
    
    v=V_rot[current_frame_number-start_frame,:]
    start_point = (obj_image_current.shape[0] // 2, obj_image_current.shape[0] // 2)
    end_point = (start_point[0] + int(v[0] * arrow_length), start_point[1] + int(v[1] * arrow_length))
    obj_image_current=cv.arrowedLine(obj_image_current, start_point, end_point, (0,255,0), 1)


    # cut border
    image_shape=obj_image_current_gray.shape
    cut_radius=r_effective
    M=koppenrath.sell_cake(r_effective,obj_image_current_gray.shape[0],obj_image_current_gray.shape[0])
    object_pixels=np.copy(obj_image_current_gray)
    object_pixels=np.float32(object_pixels[M==1])
    label=half_shelf_cluster.label(object_pixels.reshape(-1),method="GMM")
    label=label+1
    object_clustered=np.zeros(image_shape,np.uint8)
    object_clustered[M==1]=label.reshape(-1)



    temp_clustered=np.zeros(obj_image_current.shape,np.uint8)
    for l in range(image_shape[0]):
        for k in range(image_shape[1]):
            if object_clustered[l,k]==1:
                temp_clustered[l,k,:]=blue_vector
            elif object_clustered[l,k]==2:
                temp_clustered[l,k,:]=green_vector
    object_clustered=temp_clustered

    #pad arrays until they have suitable shape
    pad_length=round(max_radius-radius)
    temp_1=np.zeros((2*(max_radius)+1,2*(max_radius)+1,3),np.uint8)
    temp_2=np.copy(temp_1)
    for i in range(3):
        temp_1[:,:,i]=np.pad(obj_image_current[:,:,i], pad_width=[(pad_length,pad_length),(pad_length,pad_length)], mode='constant')
        temp_2[:,:,i]=np.pad(object_clustered[:,:,i], pad_width=[(pad_length,pad_length),(pad_length,pad_length)], mode='constant')

    temp_im_1=np.zeros((temp_1.shape[0]*b_length,temp_1.shape[1]*b_length,3),np.uint8)
    temp_im_2=np.copy(temp_im_1)
    for i in range(3):
        temp_im_1[:,:,i]=np.kron(temp_1[:,:,i],B)
        temp_im_2[:,:,i]=np.kron(temp_2[:,:,i],B)
    temp_im=np.zeros((height,width,3),np.uint8)
    temp_im[:,:temp_im_1.shape[0],:]=temp_im_1
    temp_im[:,temp_im_1.shape[0]:,:]=temp_im_2
    if annotate==True:
        temp_im=cv.putText(temp_im, '#'+str(current_frame_number).zfill(3), org, font, 
                    fontScale, color, thickness, cv.LINE_AA)
    video.write(np.uint8(temp_im))
cv.destroyAllWindows()
video.release()
